In [19]:
import tensorflow as tf

import numpy as np
import os
import time

In [20]:
path_to_file = "nosleep.txt"
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
# The unique characters in the file
vocab = sorted(set(text))

Length of text: 11981986 characters


In [21]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))
  
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

'His Tinder profile said he was 45, but he looked to be in his early thirties at most. \r\n\r\n*Looking fo'
'r a sugar baby. $700 weekly. No sex.* \r\n\r\nIt sounded too good to be true, but, as a broke university '
'student, I was willing to take my chances. I swiped right, and Tinder let me know it was a match. His'
' message came seconds later. \r\n\r\n**Hey, there sweetheart :)**\r\n\r\nI cringed at that word, I hated it, '
'but seven hundred dollars was seven hundred dollars, so I sucked it up and replied. \r\n\r\n**Hey ;)** \r\n'


In [22]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [25]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           46592     
_________________________________________________________________
gru_3 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 182)           186550    
Total params: 4,171,446
Trainable params: 4,171,446
Non-trainable params: 0
_________________________________________________________________


In [26]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [31]:
model.compile(optimizer='adam', loss=loss)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
EPOCHS=10

In [33]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 1853 steps
Epoch 1/10
1853/1853 [==============================] - 237s 128ms/step - loss: 1.5187
Epoch 2/10
1853/1853 [==============================] - 231s 125ms/step - loss: 1.2540
Epoch 3/10
1853/1853 [==============================] - 235s 127ms/step - loss: 1.2045
Epoch 4/10
1853/1853 [==============================] - 347s 187ms/step - loss: 1.1792
Epoch 5/10
 333/1853 [====>.........................] - ETA: 10:34 - loss: 1.1717

In [13]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [14]:
print(generate_text(model, start_string=u"Today I"))

Today I hatom popsicle that I had no means of being on what they are some billions in an anged bear. Parents led me with the upon humour. I followed a warm autopolf the people of several guards. I had to stop how I wished in my office.

“Well, Demous here in all friend!” I crossed, “You didn’t smile the pressure and left.

I shrugged, and to witness in. To hope you said I’m here for the first thing I’m being harder, I’ll cry, holding him up. She wast present death. You have no doubt we’ve post, straight acres felt drunk for stars. 

"Get there. Don't tell hell!", Mom appeared blaker. Burded ut an anyone first, of coughing the beautiful, don't you pick out at the night and said I was shocked, and not a picture I reached for a pillow. 
 
“McC have, I insistand that.  It’s all your neck so I awoke to the other ruins of missing terrifying throat with mudalli, Inside the cerior, dirty fiends around her. Where dark town, I stwere exposure a nice funny?"



"Ms. McKinley isn’t neclaring 
